In [ ]:
from google.colab import drive 

drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
%cd /content/gdrive/My Drive/Colab Notebooks/BAMTGAN_V1/

/content/gdrive/My Drive/Colab Notebooks/BAMTGAN_V1


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib as mlp

%matplotlib inline

from sklearn.cluster import KMeans

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
# import machine learning algorithms
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier

##Augmentator

In [ ]:
def pca_visiual(df):
  X = df.iloc[:,:-1]
  y = df.iloc[:,-1]
  y_color = len(y.unique())
  X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=10)
  scaler = StandardScaler()
  result = scaler.fit_transform(X)
  data_scaled = pd.DataFrame(result, columns=X.columns)
  data_scaled.describe()

  pca = PCA(n_components=2)
  result = pca.fit_transform(data_scaled)

  result = pd.DataFrame(result, columns=["x", "y"])
  

  target = pd.DataFrame(columns=['type'])
  target['type'] = y
  merged = pd.concat([result, target], axis=1)


  plt.figure(figsize=(16, 9))
  sns.set_palette(sns.color_palette("muted"))

  sns.scatterplot(merged['x'], merged['y'],hue=merged['type'], s=100,palette=sns.color_palette('muted', n_colors=y_color))
  plt.title("dataset")
  plt.show()

In [ ]:
def cluster_kmeans(df):
  n_clusters=4
  sc = StandardScaler()
  cc_scaled = sc.fit_transform(df)
  pd.DataFrame(cc_scaled)

  kmeans = KMeans(n_clusters,random_state=0)
  clusters = kmeans.fit(cc_scaled)

  df['cluster'] = clusters.labels_
  #df.head()

  df.groupby('cluster').count()

  
  #pca_visiual(df)

  return df

In [ ]:
def augmentator(df):
  #df = pd.read_csv(df)
  #원본 데이터 성능 평가
  print("---원본 데이터 성능평가---")
  #ml_eval(df)

  X = df.iloc[:,:-1]
  y = df.iloc[:,-1]
  X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=10)
  

  #1차 SMOTE(레이블 균형 맞추기)
  smote = SMOTE(random_state=0)
  X_train_label,y_train_label = smote.fit_resample(X_train,y_train)

  print('원본(SMOTE 적용 전) 학습용 피처/레이블 데이터 세트: ', X_train.shape, y_train.shape)
  print('1차 SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_train_label.shape, y_train_label.shape)
  print('1차 SMOTE 적용 후 레이블 값 분포: \n', pd.Series(y_train_label).value_counts())
  df_smote_label = pd.concat([X_train_label, y_train_label], axis=1)


  #2차 SMOTE(클러스터링)
  df_smote_clu = cluster_kmeans(df_smote_label)

  X = df_smote_clu.iloc[:,:-1]
  y = df_smote_clu.iloc[:,-1]
  X_train_label, X_test_label, y_train_label, y_test_label = train_test_split(X,y,test_size=0.25,random_state=10)
  smote = SMOTE(random_state=0)
  X_train_clu,y_train_clu = smote.fit_resample(X_train_label,y_train_label)

  print('1차 SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_train.shape, y_train.shape)
  print('2차 SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_train_clu.shape, y_train_clu.shape)
  print('2차 SMOTE 적용 후 레이블 값 분포: \n', pd.Series(y_train_clu).value_counts())
  df_smote_cluster = pd.concat([X_train_clu, y_train_clu], axis=1)
 

  #augmentator 후 데이터 성능 평가
  print("---augmentator 후 데이터 성능 평가---")

  X = df_smote_cluster.iloc[:,:-1]
  y = df_smote_cluster.iloc[:,-1]
  X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=10)
  
  print("이상치 제거 후 shape:",df_smote_cluster.shape)

  df_smote_cluster = df_smote_cluster.drop(columns='cluster',axis=1)
  #ml_eval(df_smote_cluster)
  
  return df_smote_cluster

##ADULT DATA

In [ ]:
adult =pd.read_csv('./data/adult_clean.csv')

In [ ]:
adult_augmentator = augmentator(adult)

---원본 데이터 성능평가---
원본(SMOTE 적용 전) 학습용 피처/레이블 데이터 세트:  (27571, 13) (27571,)
1차 SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (37616, 13) (37616,)
1차 SMOTE 적용 후 레이블 값 분포: 
 0    18808
1    18808
Name: income, dtype: int64


/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


1차 SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (27571, 13) (27571,)
2차 SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (60640, 14) (60640,)
2차 SMOTE 적용 후 레이블 값 분포: 
 1    15160
2    15160
3    15160
0    15160
Name: cluster, dtype: int64
---augmentator 후 데이터 성능 평가---
이상치 제거 후 shape: (60640, 15)


In [ ]:
adult_augmentator

,age,workclass,fnlwgt,education,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,58,6,195835,15,0,3,1,4,1,0,0,45,39,0
1,35,4,215462,11,3,6,0,4,0,4520,0,43,39,1
2,22,0,289405,15,4,0,3,4,1,0,0,40,39,0
3,41,6,408498,11,2,4,0,4,1,0,0,50,39,0
4,34,4,164280,0,2,6,0,4,1,0,0,40,32,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60635,51,2,204748,15,0,2,1,4,0,2382,0,37,39,0
60636,31,1,234755,13,0,2,3,3,0,0,0,36,37,0
60637,47,1,165448,13,2,9,2,4,0,4033,0,36,39,0
60638,51,1,182663,10,2,1,0,4,0,0,0,41,39,1


In [ ]:
#adult_augmentator.to_csv('./data/adult_augmentator.csv',index=False)